# <span style="color:green">Formation au Burkina Faso 2022</span> - Initiation à l’analyse de données Minion pour l'analyse de métagénome viraux

Created by J. Orjuela (DIADE-IRD), D. Filloux (PHIM-CIRAD) and A. Comte (PHIM-IRD) 

Septembre 2022

***

# <span style="color: #006E7F">Table of contents</span>
<a class="anchor" id="home"></a>
   

[TP3 : TAXONOMIC ASSIGNATION OF READS](#tp3) 

[1. Use Diamond for taxonomic assignation](#diamond)

   * [1.1. Download Genomic viral bank](#viraldbdiamond)
   * [1.2. Create Diamond Database](#diamondmakedb)
   * [1.3. Lauch Diamond](#rundiamond) 
   
[2. Use KAIJU for taxonomic assignation](#kraken2)
   * [3.1 Create Kaiju viruses database](#kaijudb)
   * [3.2 Launch Kaiju](#kaiju)  
   * [3.3. Adding taxa names to output file<)](#kaijunames) 
   * [3.4 Creating input file for Krona)](#kronainput) 

[3. (BONUS) Use KRAKEN2 for taxonomic assignation](#kraken2)
   * [3.1. Download a viral database](#viraldb)
   * [3.2. Run Kraken](#kraken)
   * [3.3. Vizualise Kraken output with Krona](#krakenkrona)
   
</span>

***


# <span style="color:#006E7F">__TP3 : TAXONOMIC ASSIGNATION OF READS__ <a class="anchor" id="tp3"></span>  


Taxonomic assignment is the process of assigning an Operational Taxonomic Unit (OTUs, that is, groups of related individuals) to sequences, that can be reads or contigs. To assign an OTU to a sequence it is compared against a database, but this comparison can be done in different ways. The comparison database in this assignment process must be constructed using complete genomes. There are many programs for doing taxonomic mapping, almost all of them follows one of the next strategies:


- BLAST: Using BLAST or DIAMOND, these mappers search for the most likely hit for each sequence within a database of genomes (i.e. mapping). This strategy is slow.

- K-mers: A genome database is broken into pieces of length k, so as to be able to search for unique pieces by taxonomic group, from lowest common ancestor (LCA), passing through phylum to species. Then, the algorithm breaks the query sequence (reads, contigs) into pieces of length k, look for where these are placed within the tree and make the classification with the most probable position.

- Markers: They look for markers of a database made a priori in the sequences to be classified and assign the taxonomy depending on the hits obtained.

https://carpentries-incubator.github.io/metagenomics/06-taxonomic/index.html

## <span style="color: #4CACBC;"> 1. Use Diamond for taxonomic assignation<a class="anchor" id="diamond"> </span>

### <span style="color: #4CACBC;"> 1.1. Download Genomic viral bank<a class="anchor" id="viraldbdiamond"> </span>

In [1]:
# create working repository
mkdir -p ~/work/SG-ONT-2022/ASSIGNATION/DIAMOND

In [2]:
# go inside repository
cd ~/work/SG-ONT-2022/ASSIGNATION/DIAMOND
pwd

/home/jovyan/work/SG-ONT-2022/ASSIGNATION/DIAMOND


In [3]:
# refseq viral database pre-dowloaded from ncbi (https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/)
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/viral.protein.faa

--2022-09-07 12:52:59--  https://itrop.ird.fr/ont-training-2022/viral.protein.faa
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186359083 (178M)
Saving to: ‘viral.protein.faa’

viral.protein.faa   100%[===================>] 177.73M  54.3MB/s    in 3.3s    

2022-09-07 12:53:03 (53.4 MB/s) - ‘viral.protein.faa’ saved [186359083/186359083]

FINISHED --2022-09-07 12:53:03--
Total wall clock time: 3.5s
Downloaded: 1 files, 178M in 3.3s (53.4 MB/s)


The database you use will determine the result you get for your data.

You can customise it by adding organism to the fasta file used.

Imagine you are searching for a lineage that was recently discovered and it is not part of the available databases. Would you find it?

### <span style="color: #4CACBC;"> 1.2. Create Diamond Database<a class="anchor" id="diamondmakedb"> </span>

In [4]:
diamond makedb --in viral.protein.faa -d viral

diamond v2.0.15.153 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 8
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: viral.protein.faa
Opening the database file...  [0s]
Loading sequences...  [1.459s]
Masking sequences...  [1.271s]
Writing sequences...  [0.221s]
Hashing sequences...  [0.078s]
Loading sequences...  [0s]
Writing trailer...  [0.012s]
Closing the input file...  [0s]
Closing the database file...  [0.001s]

Database sequences  591092
  Database letters  141537030
     Database hash  53675924d8042f04d5477877c942426d
        Total time  3.045000s


### <span style="color: #4CACBC;"> 1.3. Lauch Diamond<a class="anchor" id="rundiamond"> </span>

In [5]:
# Complete the command line below
diamond blastx --outfmt 6 stitle qtitle pident length mismatch gapopen qstart qend sstart send evalue bitscore ....

In [6]:
# observer les hits dans la database qui ont eu le plus de correspondance dans les données:
awk -F '\t' '{print $1}' diamond-matches.csv | sort | uniq -c | sort -n | tail -20

   1435 YP_009352885.1 coat protein [Arracacha virus V]
   1516 YP_009664759.1 coat protein [Actinidia virus A]
   1530 YP_009389466.1 coat protein [Grapevine virus K]
   1546 YP_009505636.1 capsid protein [Grapevine virus D]
   1553 YP_009551970.1 Coat protein [Grapevine virus J]
   2168 NP_619662.1 putative replicase [Grapevine virus A]
   2235 YP_009465945.1 replicase [Grapevine virus I]
   2282 YP_009551946.1 replicase [Grapevine virus G]
   2284 YP_009552539.1 replicase [Grapevine virus G]
   2290 YP_009352883.1 replicase [Arracacha virus V]
   2314 YP_009552718.1 replicase [Blackberry virus A]
   2330 YP_009551905.1 putative replicase [Grapevine virus H]
   2334 YP_006590065.1 replicase [Grapevine virus F]
   2455 YP_009664756.1 putative replicase, partial [Actinidia virus A]
   2537 YP_002117775.1 replicase [Grapevine virus E]
   2582 YP_009551967.1 Replicase [Grapevine virus J]
   2702 YP_004935358.1 ORF1 gene product [Actinidia virus B]
   2719 NP_619654.1 putative replicase [

**Observe the csv file results and comment it.**

Be careful of the separator --> choose tab

## <span style="color: #4CACBC;"> 2. Use KAIJU for taxonomic assignation <a class="anchor" id="kaiju"> </span>

Kaiju is a program for the taxonomic classification of high-throughput sequencing reads, e.g., Illumina or Roche/454, from whole-genome sequencing of metagenomic DNA. Reads are directly assigned to taxa using the NCBI taxonomy and a reference database of protein sequences from microbial and viral genomes.

Kaiju can be also used via a web server. https://kaiju.binf.ku.dk/server

In [7]:
# create working repository
mkdir ~/work/SG-ONT-2022/ASSIGNATION/KAIJU

In [8]:
cd ~/work/SG-ONT-2022/ASSIGNATION/KAIJU

### <span style="color: #4CACBC;"> 2.1 Create Kaiju viruses database<a class="anchor" id="kaijudb"> </span>

In [9]:
kaiju-makedb -s viruses

.listing                [ <=>                ]   1.76K  --.-KB/s    in 0.009s  
2022-09-07 12:53:42 URL: ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz [1800] -> ".listing" [1]
taxdump.tar.gz      100%[===================>]  56.14M  22.3MB/s    in 2.5s    
2022-09-07 12:53:45 URL: ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz [58862929] -> "taxdump.tar.gz" [1]
Extracting taxdump.tar.gz
Extracting protein sequences from downloaded files
Creating Borrows-Wheeler transform
# infilename= viruses/kaiju_db_viruses.faa
# outfilename= viruses/kaiju_db_viruses
# Alphabet= ACDEFGHIKLMNPQRSTVWY
# nThreads= 5
# length= 0.000000
# checkpoint= 3
# caseSens=OFF
# revComp=OFF
# term= *
# revsort=OFF
# help=OFF
Sequences read time = 1.057038s
SLEN 145872300
NSEQ 586242
ALPH *ACDEFGHIKLMNPQRSTVWY
SA NCHECK=1
Sorting done,  time = 91.445010s
Creating FM-Index
# filenm= viruses/kaiju_db_viruses
# removecmd= NULL (null)
# help=OFF
Reading BWT from file viruses/kaiju_db_viruses.bwt ... DONE

### <span style="color: #4CACBC;"> 2.2 Launch Kaiju (a little bit long to run)<a class="anchor" id="kaiju"> </span>

In [10]:
kaiju --help

kaiju: invalid option -- '-'
Kaiju 1.9.0
Copyright 2015-2022 Peter Menzel, Anders Krogh
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>

Usage:
   kaiju -t nodes.dmp -f kaiju_db.fmi -i reads.fastq [-j reads2.fastq]

Mandatory arguments:
   -t FILENAME   Name of nodes.dmp file
   -f FILENAME   Name of database (.fmi) file
   -i FILENAME   Name of input file containing reads in FASTA or FASTQ format

Optional arguments:
   -j FILENAME   Name of second input file for paired-end reads
   -o FILENAME   Name of output file. If not specified, output will be printed to STDOUT
   -z INT        Number of parallel threads for classification (default: 1)
   -a STRING     Run mode, either "mem"  or "greedy" (default: greedy)
   -e INT        Number of mismatches allowed in Greedy mode (default: 3)
   -m INT        Minimum match length (default: 11)
   -s INT        Minimum match score in Greedy mode (default: 65)
   -E FLOAT      Minimum E-value in Greedy mode (default

: 1

In [11]:
kaiju -t nodes.dmp -z 4 -f viruses/kaiju_db_viruses.fmi -i ~/work/SG-ONT-2022/CLEANING/reads_vs_ananas_unmapped.fastq -v -o kaiju.out

Parameters: 
  run mode: Greedy
  minimum match length: 11
  seed length: 7
  minimum blosum62 score for matches: 65
  minimum E-value: 0.01
  max number of mismatches within a match: 3
  input file 1: /home/jovyan/work/SG-ONT-2022/CLEANING/reads_vs_ananas_unmapped.fastq
  output file: kaiju.out
12:56:56 Reading database
 Reading taxonomic tree from file nodes.dmp
 Reading index from file viruses/kaiju_db_viruses.fmi
12:56:58 Start classification using 4 threads.
12:58:53 Finished.


**output format**

Kaiju will print one line for each read or read pair. The default output format contains three columns separated by tabs. Using the option -v enables the verbose output, which will print additional columns:

- either C or U, indicating whether the read is classified or unclassified.
- name of the read
- NCBI taxon identifier of the assigned taxon
- the length or score of the best match used for classification
- the taxon identifiers of all database sequences with the best match
- the accession numbers of all database sequences with the best match
- matching fragment sequence(s)

In [12]:
head kaiju.out

U	799ec77c-6555-4b9f-99a3-e58c9fbc1265	0
U	b2388cec-c33d-4a6b-948f-4cb151194e5f	0
U	fca1007f-2916-4f17-a306-3580d6af5c94	0
U	618b90fd-5af8-4669-971d-c2bde8126b46	0
U	4eefd82d-9626-4d3b-96cc-d35f4af7bbd3	0
U	07e2519f-6790-4fcd-b09e-d97d06caa3a8	0
U	8fe885e1-f363-4c55-974d-90759029f97e	0
U	176fd514-f66f-419d-84ea-7214fe64b627	0
U	74615828-dca5-476c-9398-35eebb6a2a4f	0
U	daa4394e-8ebb-4416-9abf-6a3af3f62774	0


### <span style="color: #4CACBC;"> 2.3. Adding taxa names to output file<a class="anchor" id="kaijunames"> </span>

In [13]:
kaiju-addTaxonNames -t nodes.dmp -n names.dmp -i kaiju.out -o kaiju.names.out

In [14]:
head kaiju.names.out

U	799ec77c-6555-4b9f-99a3-e58c9fbc1265	0
U	b2388cec-c33d-4a6b-948f-4cb151194e5f	0
U	fca1007f-2916-4f17-a306-3580d6af5c94	0
U	618b90fd-5af8-4669-971d-c2bde8126b46	0
U	4eefd82d-9626-4d3b-96cc-d35f4af7bbd3	0
U	07e2519f-6790-4fcd-b09e-d97d06caa3a8	0
U	8fe885e1-f363-4c55-974d-90759029f97e	0
U	176fd514-f66f-419d-84ea-7214fe64b627	0
U	74615828-dca5-476c-9398-35eebb6a2a4f	0
U	daa4394e-8ebb-4416-9abf-6a3af3f62774	0


### <span style="color: #4CACBC;"> 3.4 Creating input file for Krona<a class="anchor" id="kronainput"> </span>

In [15]:
kaiju2krona -t nodes.dmp -n names.dmp -i kaiju.out -o kaiju.out.krona

In [16]:
ktImportText -o kaiju.out.html kaiju.out.krona

tput: unknown terminal "unknown"
Writing kaiju.out.html...


**Observe the results**

Now open the HTML file by clicking on it on the left menu.

If you have an error : "Javascript must be enabled to view this page", please click on "trust HTML".

What can you see on this Krona?

We are interested in **vitiviruses**. Try to zoom in on this genus.

## <span style="color: #4CACBC;"> 3. (BONUS) Use KRAKEN2 for taxonomic assignation<a class="anchor" id="kraken2"> </span>

Kraken is a taxonomic sequence classifier that assigns taxonomic labels to DNA sequences. Kraken examines the k-mers within a query sequence and uses the information within those k-mers to query a database. That database maps k-mers to the lowest common ancestor (LCA) of all genomes known to contain a given k-mer.

In [17]:
kraken2 --help

Usage: kraken2 [options] <filename(s)>

Options:
  --db NAME               Name for Kraken 2 DB
                          (default: none)
  --threads NUM           Number of threads (default: 1)
  --quick                 Quick operation (use first hit or hits)
  --unclassified-out FILENAME
                          Print unclassified sequences to filename
  --classified-out FILENAME
                          Print classified sequences to filename
  --output FILENAME       Print output to filename (default: stdout); "-" will
                          suppress normal output
  --confidence FLOAT      Confidence score threshold (default: 0.0); must be
                          in [0, 1].
  --minimum-base-quality NUM
                          Minimum base quality used in classification (def: 0,
                          only effective with FASTQ input).
  --report FILENAME       Print a report with aggregrate counts/clade to file
  --use-mpa-style         With --report, format report output

### <span style="color: #4CACBC;"> 3.1. Download a viral database<a class="anchor" id="viraldb"> </span>

For this TP we will download a pre-made simplified kraken database.

MiniKraken DB_8GB (6.0 GB): A pre-built 8 GB database constructed from complete bacterial, archaeal, and viral genomes in RefSeq (as of Oct. 18, 2017). This can be used by users without the computational resources needed to build a Kraken database. This contains around 5% of kmers from the original standard database. 
It can be found here: https://ccb.jhu.edu/software/kraken/

You can build your own custom Database (see https://github.com/DerrickWood/kraken2/wiki/Manual). However, it take a lot of ressources and time.


In [18]:
# create working repository
mkdir -p ~/work/SG-ONT-2022/ASSIGNATION/KRAKEN

In [19]:
# run Kraken
cd ~/work/SG-ONT-2022/ASSIGNATION/KRAKEN

In [20]:
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/minikraken2_v2_8GB_201904.tgz

--2022-09-07 13:06:22--  https://itrop.ird.fr/ont-training-2022/minikraken2_v2_8GB_201904.tgz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5935990636 (5.5G) [application/x-gzip]
Saving to: ‘minikraken2_v2_8GB_201904.tgz’

minikraken2_v2_8GB_ 100%[===================>]   5.53G  59.8MB/s    in 1m 40s  

2022-09-07 13:08:02 (56.6 MB/s) - ‘minikraken2_v2_8GB_201904.tgz’ saved [5935990636/5935990636]

FINISHED --2022-09-07 13:08:02--
Total wall clock time: 1m 40s
Downloaded: 1 files, 5.5G in 1m 40s (56.6 MB/s)


In [21]:
#uncompress the database
tar zxvf minikraken2_v2_8GB_201904.tgz

minikraken2_v2_8GB_201904_UPDATE/
minikraken2_v2_8GB_201904_UPDATE/taxo.k2d
minikraken2_v2_8GB_201904_UPDATE/opts.k2d
minikraken2_v2_8GB_201904_UPDATE/database100mers.kmer_distrib
minikraken2_v2_8GB_201904_UPDATE/database150mers.kmer_distrib
minikraken2_v2_8GB_201904_UPDATE/database200mers.kmer_distrib
minikraken2_v2_8GB_201904_UPDATE/hash.k2d


In [22]:
# Inspect the database content
kraken2-inspect --db minikraken2_v2_8GB_201904_UPDATE/ | head -15

# Database options: nucleotide db, k = 35, l = 31
# Spaced mask = 11111111111111111111111111111111111111001100110011001100110011
# Toggle mask = 1110001101111110001010001100010000100111000110110101101000101101
# Total taxonomy nodes: 21114
# Table size: 1399930605
# Table capacity: 2000000000
# Min clear hash value = 14207118309059100672
100.00	1399930605	619083	R	1	root
 98.96	1385354760	72724	R1	131567	  cellular organisms
 83.11	1163416379	1025603	D	2	    Bacteria
 42.38	593288710	1473535	P	1224	      Proteobacteria
 20.38	285373440	759477	C	1236	        Gammaproteobacteria
  5.69	79638023	3551	O	72274	          Pseudomonadales
  4.72	66079245	52184	F	135621	            Pseudomonadaceae
  4.64	64952993	11121851	G	286	              Pseudomonas


### <span style="color: #4CACBC;"> 3.2. run Kraken<a class="anchor" id="kraken"> </span>

In [23]:
kraken2 --db minikraken2_v2_8GB_201904_UPDATE/ ../../CLEANING/reads_vs_ananas_unmapped.fastq --report report.txt --report-minimizer-data --> output_kraken

Loading database information... done.
406090 sequences (174.21 Mbp) processed in 13.736s (1773.8 Kseq/m, 760.92 Mbp/m).
  203844 sequences classified (50.20%)
  202246 sequences unclassified (49.80%)


**Standard Kraken Output Format**

Each sequence (or sequence pair, in the case of paired reads) classified by Kraken 2 results in a single line of output. Kraken 2's output lines contain five tab-delimited fields; from left to right, they are:

- "C"/"U": a one letter code indicating that the sequence was either classified or unclassified.

- The sequence ID, obtained from the FASTA/FASTQ header.

- The taxonomy ID Kraken 2 used to label the sequence; this is 0 if the sequence is unclassified.

- The length of the sequence in bp. In the case of paired read data, this will be a string containing the lengths of the two sequences in bp, separated by a pipe character, e.g. "98|94".

- A space-delimited list indicating the LCA mapping of each k-mer in the sequence(s). For example, "562:13 561:4 A:31 0:1 562:3" would indicate that:

        - the first 13 k-mers mapped to taxonomy ID #562
        - the next 4 k-mers mapped to taxonomy ID #561
        - the next 31 k-mers contained an ambiguous nucleotide
        - the next k-mer was not in the database
        - the last 3 k-mers mapped to taxonomy ID #562


In [24]:
head output_kraken

C	799ec77c-6555-4b9f-99a3-e58c9fbc1265	1491	335	0:217 1491:4 0:21 1491:2 0:6 1491:5 0:16 1491:4 2:5 0:20 9606:1
U	37c0c305-d935-4b3b-b336-24b4c4c8021d	0	332	0:56 9606:2 0:240
U	02df7f95-9bbe-4b55-9c8b-78955e3d9210	0	208	0:174
U	b86266e6-4b84-4ed6-abde-302e336f6c24	0	429	0:53 9606:5 0:337
U	b7f946d2-7f1d-492c-b187-3ebc0770a15c	0	292	0:222 131567:2 0:34
U	c2615778-aa7c-4906-8c53-75cd9fa196f9	0	605	0:571
C	b2388cec-c33d-4a6b-948f-4cb151194e5f	1491	417	0:42 9606:1 0:230 1491:1 0:7 1491:2 0:14 1491:5 0:39 1491:1 1239:3 0:38
U	26f48c03-b0cd-485e-8e6e-6cd4d9be7b74	0	245	0:211
U	515b1cfe-bfc5-4595-966f-ecf1c09a0b12	0	595	0:561
U	97e06d49-a952-4adf-b223-8a721c1b48b7	0	253	0:33 9606:2 0:184


**Report format output**

The format with the --report-minimizer-data flag, then, is similar to that described in [Sample Report Output Format], but slightly different. The fields in this new format, from left-to-right, are:

- 1. Percentage of fragments covered by the clade rooted at this taxon
- 2. Number of fragments covered by the clade rooted at this taxon
- 3. Number of fragments assigned directly to this taxon
- 4. Number of minimizers in read data associated with this taxon (new)
- 5. An estimate of the number of distinct minimizers in read data associated with this taxon (new)
- 6. A rank code, indicating (U)nclassified, (R)oot, (D)omain, (K)ingdom, (P)hylum, (C)lass, (O)rder, (F)amily, (G)enus, or (S)pecies. Taxa that are not at any of these 10 ranks have a rank code that is formed by using the rank code of the closest ancestor rank with a number indicating the distance from that rank. E.g., "G2" is a rank code indicating a taxon is between genus and species and the grandparent taxon is at the genus rank.
- 7. NCBI taxonomic ID number
- 8. Indented scientific name


In [25]:
head -10 report.txt

 49.80	202246	202246	0	0	U	0	unclassified
 50.20	203844	29	948365	67656	R	1	root
 49.89	202611	2948	940454	67451	R1	131567	  cellular organisms
 25.37	103027	1620	377336	12115	D	2	    Bacteria
 24.06	97702	36	285250	3968	D1	1783272	      Terrabacteria group
 23.75	96453	58	274840	1157	P	1239	        Firmicutes
 23.70	96235	2	270308	427	C	186801	          Clostridia
 23.70	96231	15	270265	401	O	186802	            Clostridiales
 23.69	96189	1	270084	279	F	31979	              Clostridiaceae
 23.69	96188	20	270062	262	G	1485	                Clostridium


### <span style="color: #4CACBC;"> 3.3. Vizualise kraken output with krona<a class="anchor" id="krakenkrona"> </span>

In [28]:
ktImportTaxonomy -m 3 -t 5 report.txt -o kraken.html 2> krakenkrona.err

Loading taxonomy...
Importing report.txt...
Writing kraken.html...
